In [2]:
import pandas as pd
pd.set_option('display.max_columns', 70)
pd.set_option('display.max_rows',150)

In [14]:
df = pd.read_csv('input/planillas/210830_f11_1.csv', sep=';', dtype='object')

In [4]:
def gr(df3, dfna):
    df = df3.copy()
    i = 0
    idf=0
    while i < dfna.shape[0]:
        i = dfna.index[idf]
        df.iloc[i-4,21] = dfna.iloc[idf-2,1:2].values[0]
        df.iloc[i-4,22:27] = dfna.iloc[idf-1,1:6].values
        df.iloc[i-4,27:49] = dfna.iloc[idf,1:23].values
        idf+=3
    return df 

In [5]:
def grdos(dfarg, dfna):
    df = dfarg.copy()
    i = 0
    idf=0
    while idf < dfna.shape[0]:
        i = dfna.index[idf]
        df.iloc[i-2,3] = df.iloc[i-1,1]
        df.iloc[i-2,4:6] = df.iloc[i,1:3].values
        df.iloc[i-2,6:21] = df.iloc[i+1,1:16].values
        idf+=1
    return df 

Limpieza parte 1 

In [15]:
dfnull = df[df.isnull().all(axis=1)]
df = df[~df.isnull().all(axis=1)].reset_index(drop=True)

Limpieza parte 2

In [16]:
dfna = df[df['Nro. F11'].isna()]
idfna = dfna.index.values 

dfnacliente = df.loc[(df['Nro. F11'].isna())&((df['F03 Asociado']=='CLIENTE')|(df['F03 Asociado']=='EMPRESA'))]
index_dfnac = dfnacliente.index

menosuno = (index_dfnac-1).isin(idfna)
menosdos = (index_dfnac-2).isin(idfna)

oklist = []
for i in range(len(index_dfnac)): 
    oklist.append((menosuno[i]==True) & (menosdos[i]==True))
    
itochange = index_dfnac[oklist]

acamb = dfnacliente.loc[itochange]

df2 = gr(df, acamb)
iuno = acamb.index.values
idos = acamb.index.values-1
itres = acamb.index.values-2
df3 = df2.loc[~df2.index.isin(iuno)]
df3 = df3.loc[~df3.index.isin(idos)]
df3 = df3.loc[~df3.index.isin(itres)]
df3.reset_index(drop=True, inplace=True)

dfuno = df.loc[(df.index.isin(iuno))|(df.index.isin(idos))|(df.index.isin(itres))]
dfuno.to_excel(f'output/df-deleted-1.xlsx', sheet_name=f'f11', index=False) 



In [18]:
#dfnacliente.to_excel(f'output/dfna-2.xlsx', sheet_name=f'f11', index=False) 

Limpieza parte 2

In [17]:
dfna = df3[df3['Nro. F11'].isna()]

indexdigit = df3.loc[(df3['Nro. F11'].isna())&(df3['F03 Asociado'].str.isdigit())].index

index_dfna = dfna.index.values
masuno = (indexdigit+1).isin(index_dfna)
menosuno = (indexdigit-1).isin(index_dfna)

oklist = []
for i in range(len(indexdigit)): 
    oklist.append((masuno[i]==True) & (menosuno[i]==True))
    
itochange = indexdigit[oklist]

In [18]:
acamb = dfna.loc[itochange]
df4 = grdos(df3, acamb)
iuno = acamb.index.values-1
idos = acamb.index.values
itres = acamb.index.values+1
df4 = df4.loc[~df4.index.isin(iuno)]
df4 = df4.loc[~df4.index.isin(idos)]
df4 = df4.loc[~df4.index.isin(itres)]
df4.reset_index(drop=True, inplace=True)

dfdos = df3.loc[(df3.index.isin(iuno))|(df3.index.isin(idos))|(df3.index.isin(itres))]
dfdos.to_excel(f'output/df-deleted-2.xlsx', sheet_name=f'f11', index=False) 

Limpieza parte 3

In [19]:
bogota = df4.loc[(df4['Nro. F11'].isna())&(df4['F03 Asociado']=='BOGOTA')]

for i in bogota.index:
    df4.iloc[i-1,22:27] = df4.iloc[i,1:6].values
    
df5 = df4[~df4.index.isin(bogota.index)].reset_index(drop=True)

cliente/empresa

In [20]:
cliente_empresa = df5.loc[(df5['Nro. F11'].isna())&((df5['F03 Asociado']=='CLIENTE')|(df5['F03 Asociado']=='EMPRESA'))]

for i in cliente_empresa.index:
    df5.iloc[i-1,27:49] = df5.iloc[i,1:23].values
    
df6 = df5[~df5.index.isin(cliente_empresa.index)].reset_index(drop=True)

direccion

In [21]:
direc = df6.loc[(~df6['F03 Asociado'].str.isdigit().fillna(False))&(df6['Nro. F11'].isna())&(df6['F03 Asociado']!='CD FALABELLA')]

for i in direc.index:
    df6.iloc[i-1,21:27] = df6.iloc[i,1:7].values
    df6.iloc[i-1,27:49] = df6.iloc[i,27:49].values
    
df7 = df6[~df6.index.isin(direc.index)].reset_index(drop=True)

Guardar archivos 

In [12]:
#df3.to_excel(f'output/df3.xlsx', sheet_name=f'f11', index=False) 
#df4.to_excel(f'output/df4.xlsx', sheet_name=f'f11', index=False) 
#df5.to_excel(f'output/df5.xlsx', sheet_name=f'f11', index=False) 
#df6.to_excel(f'output/df6.xlsx', sheet_name=f'f11', index=False) 
#df7.to_excel(f'output/df7-pap-0823.xlsx', sheet_name=f'f11', index=False) 

In [26]:
df7 = pd.read_excel('input/db-pbi/f11/df7.xlsx', dtype='object')

In [22]:
df7.to_csv('input/db-pbi/f11/210830-df11_2.csv', sep=';', index=False)

# F11 directriz

In [23]:
emp = pd.read_excel('input/dirf11/empleados.xlsx', sheet_name='db')

In [24]:
usu = pd.read_excel('input/dirf11/usuarios.xlsx', sheet_name='db')

In [26]:
f11 = pd.read_csv('input/dirf11/210830-df11.csv', sep=';', dtype='object')

In [27]:
f11.head()

,Nro. F11,F03 Asociado,Nombre Cliente,Apellido Cliente,RUT Cliente,Localidad,Local,Fecha Creacisn,Usuario Creacisn,Nro Confeccisn,Fecha,Terminal,Secuencia,Ticket,Vendedor,Tipo Servicio,RUT ST,Razsn Social,Marca,Desc. Marca,Sucursal,Direccisn,Comuna,Ciudad,Fono,Fax,Atencisn,Dueqo,Fecha Pactada,UPC,Linea,Subclase,SKU,Descripcisn,Cantidad,Nro Serie,Precio,Fecha Despacho,Usuario Despacho,Fecha Entrega,Usuario Entrega,Fecha Retiro,Usuario Retiro,Estado,Sub Estado,Cant.Impresisn,O.Trabajo,Presupuesto,Observacion,Patente,Rut Transp,Nombre Transp
0,1111092038,,CARLOS SIMON,CHIPIAJE BAUTISTA,1121874644,VILLAVICENCIO VILLAVICENCIO,CD FALABELLA,30/08/2021,NCANTORB,NaN,NaN,NaN,1,NaN,NaN,NaN,890100577,DEPRISA,VIVO,VIVO,DEPRISA EXTRAVIADOS PV,AV EL DORADO # 59-15,BOGOTA,BOGOTA,3226101908,NaN,EDUARDO LUIS CAICEDO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1111092021,,LUIS EDUARDO,MEJ�A VILLA,79491824,BOGOTA BOGOTA,CD FALABELLA,30/08/2021,NCANTORB,NaN,NaN,NaN,1,NaN,NaN,NaN,830051440,EXXE LOG�STICA S.A.S,PYROREY,PYROREY,EXXE EXTRAVIADOS PV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1111092014,,HERNAN,SALAZAR,71318921,MEDELLIN MEDELLIN,CD FALABELLA,30/08/2021,NCANTORB,NaN,NaN,NaN,1,NaN,NaN,NaN,860016819,AL DIA LOGISTICA,LG,LG,ALDIA AVERIAS PV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1111092007,,LAURA,LOPEZ,1032479444,BOGOTA BOGOTA,CD FALABELLA,30/08/2021,NCANTORB,NaN,NaN,NaN,1,NaN,NaN,NaN,860512330,SERVIENTREGA S.A,SOFIA,SOFIA,SERVIENTREGA EXTRAVIADOS PV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1111091994,,MARIA DEL CARME,ROJAS,52550784,BOGOTA BOGOTA,CD FALABELLA,30/08/2021,NCANTORB,NaN,NaN,NaN,1,NaN,NaN,NaN,900632226,MENSAJEROS URBANOS,AMERICAN T,AMERICAN TOURISTER,MENSAJEROS URBANOS ENTREGAS F PV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
f11usua = f11.merge(usu, how='left', left_on='Usuario Creacisn', right_on='Nombre Usuario')

In [29]:
f11usua[f11usua['Nombre Usuario'].isna()]['Usuario Creacisn'].unique()

array([nan, 'ASOLARE', '29/07/2021', '12/08/2021'], dtype=object)

In [30]:
usu.drop_duplicates('Nombre Usuario', inplace=True)

In [31]:
usu.head()

,Usuario,Nombre Usuario,Rut,Fecha Estado,Organizacion,Grupos,Estado,Fecha Creacion,Fecha ultimo Ingreso,Unnamed: 9
0,AARON DAVID AVILA,ADAVILA,1016006055,2019-09-19 00:00:00,CD,FALCO CD RECIBO,ACTIVO,2019-09-19 08:43:02.0,sin registro,NaN
5,AARON FELIPE CAMACHO,AFCAMACHO,80876303,2017-08-01 00:00:00,SANTA FE,,FINIQUITADO,2013-08-13 11:33:29.0,sin registro,NaN
6,AASSENETH PATRICIA ROLDAN,AROLDAN,1019095340,2018-06-24 00:00:00,VEMPRESA,FALCO VE ASISTENTE,DESACTIVADO,2017-07-24 17:13:33.0,sin registro,NaN
7,AASSENETH PATRICIA ROLDON,AROLDON,101995340,2017-07-14 00:00:00,ADMINIST,FALCO VE ASISTENTE,FINIQUITADO,2017-06-13 09:50:48.0,sin registro,NaN
8,ABEL LOZANO,ALOZANOU,1075626288,2020-09-10 00:00:00,SUBA,FALCO ASESOR SAC,DESACTIVADO,2020-08-10 11:15:16.0,sin registro,NaN


In [33]:
emp.drop_duplicates('Num_Documento', inplace=True)

In [34]:
f11res = f11usua.merge(emp, how='left', left_on='Rut', right_on='Num_Documento')

In [35]:
f11res.shape

(5233, 68)

In [36]:
f11res[f11res['Num_Documento'].isna()]['Nombre Usuario'].unique()

array(['ANDLEON', 'MROJASY', 'GPUERTAS', nan, 'YHERNANDC', 'FPEDRAZR',
       'JINFANPO'], dtype=object)

In [37]:
f11res.head()

,Nro. F11,F03 Asociado,Nombre Cliente,Apellido Cliente,RUT Cliente,Localidad,Local,Fecha Creacisn,Usuario Creacisn,Nro Confeccisn,Fecha,Terminal,Secuencia,Ticket,Vendedor,Tipo Servicio,RUT ST,Razsn Social,Marca,Desc. Marca,Sucursal_x,Direccisn,Comuna,Ciudad,Fono,Fax,Atencisn,Dueqo,Fecha Pactada,UPC,Linea,Subclase,SKU,Descripcisn,Cantidad,Nro Serie,Precio,Fecha Despacho,Usuario Despacho,Fecha Entrega,Usuario Entrega,Fecha Retiro,Usuario Retiro,Estado,Sub Estado,Cant.Impresisn,O.Trabajo,Presupuesto,Observacion,Patente,Rut Transp,Nombre Transp,Usuario,Nombre Usuario,Rut,Fecha Estado,Organizacion,Grupos,Estado,Fecha Creacion,Fecha ultimo Ingreso,Unnamed: 9,Cod_Empleado,Num_Documento,nombre completo,Sucursal_y,Cargo,Tipo
0,1111092038,,CARLOS SIMON,CHIPIAJE BAUTISTA,1121874644,VILLAVICENCIO VILLAVICENCIO,CD FALABELLA,30/08/2021,NCANTORB,NaN,NaN,NaN,1,NaN,NaN,NaN,890100577,DEPRISA,VIVO,VIVO,DEPRISA EXTRAVIADOS PV,AV EL DORADO # 59-15,BOGOTA,BOGOTA,3226101908,NaN,EDUARDO LUIS CAICEDO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NESTOR CANTOR,NCANTORB,1012346396,2020-09-18 00:00:00,FALCOL,FALCO CD ANALISTA DESPACHOS,ACTIVO,2020-09-18 12:37:50.0,sin registro,NaN,119040,1.012346e+09,NELSON ANTONIO CANTOR BELTRAN,FALABELLA - CENTRO DE DISTRIBUCION,ANALISTA POSTVENTA,planta
1,1111092021,,LUIS EDUARDO,MEJ�A VILLA,79491824,BOGOTA BOGOTA,CD FALABELLA,30/08/2021,NCANTORB,NaN,NaN,NaN,1,NaN,NaN,NaN,830051440,EXXE LOG�STICA S.A.S,PYROREY,PYROREY,EXXE EXTRAVIADOS PV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NESTOR CANTOR,NCANTORB,1012346396,2020-09-18 00:00:00,FALCOL,FALCO CD ANALISTA DESPACHOS,ACTIVO,2020-09-18 12:37:50.0,sin registro,NaN,119040,1.012346e+09,NELSON ANTONIO CANTOR BELTRAN,FALABELLA - CENTRO DE DISTRIBUCION,ANALISTA POSTVENTA,planta
2,1111092014,,HERNAN,SALAZAR,71318921,MEDELLIN MEDELLIN,CD FALABELLA,30/08/2021,NCANTORB,NaN,NaN,NaN,1,NaN,NaN,NaN,860016819,AL DIA LOGISTICA,LG,LG,ALDIA AVERIAS PV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NESTOR CANTOR,NCANTORB,1012346396,2020-09-18 00:00:00,FALCOL,FALCO CD ANALISTA DESPACHOS,ACTIVO,2020-09-18 12:37:50.0,sin registro,NaN,119040,1.012346e+09,NELSON ANTONIO CANTOR BELTRAN,FALABELLA - CENTRO DE DISTRIBUCION,ANALISTA POSTVENTA,planta
3,1111092007,,LAURA,LOPEZ,1032479444,BOGOTA BOGOTA,CD FALABELLA,30/08/2021,NCANTORB,NaN,NaN,NaN,1,NaN,NaN,NaN,860512330,SERVIENTREGA S.A,SOFIA,SOFIA,SERVIENTREGA EXTRAVIADOS PV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NESTOR CANTOR,NCANTORB,1012346396,2020-09-18 00:00:00,FALCOL,FALCO CD ANALISTA DESPACHOS,ACTIVO,2020-09-18 12:37:50.0,sin registro,NaN,119040,1.012346e+09,NELSON ANTONIO CANTOR BELTRAN,FALABELLA - CENTRO DE DISTRIBUCION,ANALISTA POSTVENTA,planta
4,1111091994,,MARIA DEL CARME,ROJAS,52550784,BOGOTA BOGOTA,CD FALABELLA,30/08/2021,NCANTORB,NaN,NaN,NaN,1,NaN,NaN,NaN,900632226,MENSAJEROS URBANOS,AMERICAN T,AMERICAN TOURISTER,MENSAJEROS URBANOS ENTREGAS F PV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NESTOR CANTOR,NCANTORB,1012346396,2020-09-18 00:00:00,FALCOL,FALCO CD ANALISTA DESPACHOS,ACTIVO,2020-09-18 12:37:50.0,sin registro,NaN,119040,1.012346e+09,NELSON ANTONIO CANTOR BELTRAN,FALABELLA - CENTRO DE DISTRIBUCION,ANALISTA POSTVENTA,planta


In [38]:
f11res[f11res['Num_Documento'].notna()].shape

(4195, 68)

In [39]:
f11res.to_excel('input/dirf11/210830-data_directriz_f11.xlsx', sheet_name='gco', index=False)